In [ ]:
# The goal is to fine tune GPT-2 on a dataset of shakespear text

In [ ]:
# If you are running this notebook on Google Colab run this cell to clone the repository
!git clone https://github.com/Memento2121/Fine-tuning-GPT2.git
%cd Fine-tuning-GPT2

Cloning into 'Fine-tuning-GPT2'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 4), reused 11 (delta 2), pack-reused 0
Receiving objects: 100% (19/19), 434.16 KiB | 912.00 KiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/Fine-tuning-GPT2


In [ ]:
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, AutoTokenizer

import os

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

from torch.cuda.amp import autocast, GradScaler

import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# Load pre-trained model and tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# We are gonna do a LoRA fineturning on the shakespear dataset

In [ ]:
!pip install peft
from peft import prepare_model_for_kbit_training

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attn.c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model.gradient_checkpointing_enable()
#model = prepare_model_for_kbit_training(model)
#model = get_peft_model(model, config)
#model = torch.compile(model)


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# Define function to print trainable parameters
def print_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params}")
    print(f"Total parameters: {total_params}")
    print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")

# Print trainable parameters
print_trainable_parameters(model)

Trainable parameters: 294912
Total parameters: 124734720
Percentage of trainable parameters: 0.24%


In [ ]:
# dataset is a text file of shakespear text

with open('input.txt', 'r') as file:
    data = file.read()

data = data[:30000]

In [ ]:
# Tokenize the dataset

dataset = tokenizer.encode(data, return_tensors='pt')

print(dataset.size())

Token indices sequence length is longer than the specified maximum sequence length for this model (9103 > 1024). Running this sequence through the model will result in indexing errors


torch.Size([1, 9103])


In [ ]:
# split the dataset into training and validation sets

n = 0.85

train_size = int(dataset.size()[1] * n)

train_dataset = dataset[:, :train_size]
val_dataset = dataset[:, train_size:]

# parameters of GPT2 model

config2 = GPT2Config.from_pretrained('gpt2')

# get the block size of the model

block_size = config2.n_positions

from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        # Subtract self.block_size + 1 to avoid going out of bounds
        return self.data.size()[1] - self.block_size - 1

    def __getitem__(self, idx):
        # Input sequence is from idx to idx+self.block_size
        input_sequence = self.data[:, idx:idx+self.block_size]
        # Target sequence is shifted by one token to the right
        target_sequence = self.data[:, idx+1:idx+self.block_size+1]
        return input_sequence, target_sequence


train_dataset = TextDataset(train_dataset, block_size)

val_dataset = TextDataset(val_dataset,
                          block_size)

train_loader = DataLoader(train_dataset,
                          batch_size=8, shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=8)

print(len(train_loader))
print(len(val_loader))


839
43


In [ ]:
print(val_dataset.data.size())
print(train_dataset.data.size())

torch.Size([1, 1366])
torch.Size([1, 7737])


In [ ]:
epochs = 2
lr = 2e-5
optimizer = AdamW(model.parameters(), lr=lr)

total_steps = len(train_loader) * epochs
print(total_steps)

1678


In [ ]:
save_path = './model_checkpoints'
os.makedirs(save_path, exist_ok=True)

In [ ]:
"""
import torch
import torch.autograd.profiler as profiler

# Profiling within the training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0

    # Start profiling
    with profiler.profile(record_shapes=True, use_cuda=True) as prof:
        for i, (input_seq, attention_mask) in enumerate(train_loader):
            if i >= 10:  # Profile only the first 10 iterations
                break

            input_seq = input_seq.to(device)
            attention_mask = attention_mask.to(device)

            # Forward pass
            with profiler.record_function("forward"):
                outputs = model(input_seq, attention_mask=attention_mask, labels=input_seq)
                loss = outputs.loss

            # Backward pass
            with profiler.record_function("backward"):
                loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()

            if i % 10 == 0:  # Adjust the logging frequency as needed
                print(f"Epoch {epoch+1}/{epochs}, Iteration {i+1}/{len(train_loader)}, Loss: {loss.item()}")

    print(f"Epoch {epoch+1} Training Loss: {total_loss / (i+1)}")

    # Export profiling results
    prof.export_chrome_trace(f"profile_epoch_{epoch+1}.json")

# Download the profiling results
filename = f"profile_epoch_{epoch+1}.json"
files.download(filename)
"""

'\nimport torch\nimport torch.autograd.profiler as profiler\n\n# Profiling within the training loop\nfor epoch in range(epochs):\n    model.train()\n    total_loss = 0\n    \n    # Start profiling\n    with profiler.profile(record_shapes=True, use_cuda=True) as prof:\n        for i, (input_seq, attention_mask) in enumerate(train_loader):\n            if i >= 10:  # Profile only the first 10 iterations\n                break\n            \n            input_seq = input_seq.to(device)\n            attention_mask = attention_mask.to(device)\n            \n            # Forward pass\n            with profiler.record_function("forward"):\n                outputs = model(input_seq, attention_mask=attention_mask, labels=input_seq)\n                loss = outputs.loss\n            \n            # Backward pass\n            with profiler.record_function("backward"):\n                loss.backward()\n            \n            optimizer.step()\n            optimizer.zero_grad()\n            \n   

In [ ]:
import time

scaler = GradScaler()

# Function to measure time per batch
def measure_time_per_batch(dataloader, num_batches=10):
    model.train()
    torch.cuda.synchronize()
    start_time = time.time()
    for i, (input_seq, target_seq) in enumerate(dataloader):
        if i >= num_batches:
            break
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
          outputs = model(input_seq, labels=target_seq)
          loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        #optimizer.zero_grad()
    torch.cuda.synchronize()
    end_time = time.time()
    avg_time_per_batch = (end_time - start_time) / num_batches
    return avg_time_per_batch

model.config.use_cache = False

# Measure the average time per batch
avg_time_per_batch = measure_time_per_batch(train_loader)
print(f"Average time per batch: {avg_time_per_batch:.4f} seconds")

# Calculate total training time
total_iterations = len(train_loader) * epochs
estimated_total_time = total_iterations * avg_time_per_batch
print(f"Estimated total training time: {estimated_total_time / 3600:.2f} hours")

Average time per batch: 0.7734 seconds
Estimated total training time: 0.36 hours


In [ ]:
# Fine-tune the model

for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_val_loss = 0
    for i, (input_seq, target_seq) in enumerate(train_loader):
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
          outputs = model(input_seq, labels=target_seq)
          loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        #optimizer.zero_grad()
        total_loss += loss.item()
        if i % 500 == 0 and i > 0:
            model.eval()
            print(f"Epoch {epoch} Iter {i} Loss: {loss.item()}")
            total_val_loss = 0
            for j, (input_seq, target_seq) in enumerate(val_loader):
                input_seq = input_seq.to(device)
                target_seq = target_seq.to(device)
                with torch.no_grad():
                    outputs = model(input_seq, labels=target_seq)
                loss = outputs.loss
                total_val_loss += loss.item()
            print(f"Epoch {epoch} Iter {i} Validation Loss: {total_val_loss/j}")
            checkpoint_path = os.path.join(save_path, f'checkpoint_epoch_{i}.pt')
            model.save_pretrained(checkpoint_path)
            tokenizer.save_pretrained(checkpoint_path)
            model.train()
    print(f"Epoch {epoch} Total Loss: {total_loss}")


KeyboardInterrupt: 

In [ ]:
#model.save_pretrained('./fine_tuned_gpt2')
#tokenizer.save_pretrained('./fine_tuned_gpt2')

# Generate text

model.eval()
prompt = "To be or not to"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)


with torch.no_grad():
  output = model.generate(
      input_ids=input_ids,
      max_length=100,
      num_return_sequences=1,
      no_repeat_ngram_size=2,
      pad_token_id=tokenizer.eos_token_id
  )

print(tokenizer.decode(output[0]))